In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import pylab as plt

from scipy import stats
import joblib
from functools import reduce

from importlib import reload
import sys
import os
import datetime

from sksurv.metrics import cumulative_dynamic_auc#,cumulative_dynamic_auprc

ImportError: cannot import name 'cumulative_dynamic_auprc' from 'sksurv.metrics' (/scratch/c.c21013066/docker/envs/py38R/lib/python3.8/site-packages/sksurv/metrics.py)

# recompute risk with trained model

In [2]:
data_path = '/scratch/c.c21013066/data/ukbiobank/analyses/survival/prodromal/noOsteo'

In [7]:
feature = 'intercept'
name = 'diag_ProdPopulationNoPD_allHC'
diag = 'diag_ProdPopulationNoPD'
cv = 0
time_points = np.arange(2.5, 7,0.1)
dt=dtype=[('Status', '?'), ('Survival_in_years', '<f8')]

In [8]:
# load preds
preds = np.load(f'{data_path}/{feature}/{name}/rsf_testpred_CV{cv}.csv.npy')
# load true
true = pd.read_csv(f'{data_path}/{feature}/{name}/rsftest_cv{cv}.csv')
# auc
auc = pd.read_csv(f'{data_path}/{feature}/{name}/rsf_aucs_5cv.csv')

In [17]:
merged = pd.read_csv(f'/scratch/c.c21013066/data/ukbiobank/merged_data/populationNoOsteoAllHC.csv').set_index('eid')
prodage = pd.read_csv(f'/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoMatchedHC.csv').set_index('eid')
merged.loc[prodage[prodage['diag_ProdHC']==1].index,'acc_time_to_diagnosis'] = prodage.loc[prodage['diag_ProdHC']==1,'acc_time_to_diagnosis'].values

merged.loc[merged['diag_ProdPopulationNoPD']==0,'acc_time_to_diagnosis'] = (pd.Timestamp(datetime.datetime(2021,3,1)) - pd.to_datetime(merged.loc[merged['diag_ProdPopulationNoPD']==0,'date_accelerometry']) )/ np.timedelta64(1,'Y')
merged['Intercept'] = 1
merged = merged.dropna(subset=[diag])

In [18]:
train = merged[ ~merged.index.isin(true.eid) ]
train[diag] = train[diag].astype('?')
train['acc_time_to_diagnosis'] = train['acc_time_to_diagnosis'].astype('<f8')
train = train.drop(columns=['Status'])
train = train.rename(columns={diag:'Status','acc_time_to_diagnosis':'Survival_in_years'})

<ipython-input-18-b4aca5380a7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[diag] = train[diag].astype('?')
<ipython-input-18-b4aca5380a7b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['acc_time_to_diagnosis'] = train['acc_time_to_diagnosis'].astype('<f8')


In [19]:
train.shape,merged.shape[0]-true.shape[0],merged.shape,true.shape

((26497, 182), 26497, (33122, 183), (6625, 85))

In [20]:
33122/100*80

26497.600000000002

In [21]:
100/5

20.0

In [22]:
data_y = np.array([tuple(row) for row in train[['Status','Survival_in_years']].values], dtype=dt)

data_y_test = np.array([tuple(row) for row in true[['Status','Survival_in_years']].values], dtype=dt)

In [23]:
train[['Status','Survival_in_years']]

,Status,Survival_in_years
eid,,
1001731,False,5.756678
1002115,False,5.485625
1002406,False,6.085226
1002485,False,5.450032
1004577,False,6.137247
...,...,...
6024129,False,6.739586
6024315,False,7.046232
6024343,False,5.899049


In [28]:
rsf = joblib.load(f'{data_path}/{feature}/{name}/modelrsf_CV{cv}.joblib')
y_riskpred = rsf.predict(true.iloc[:,3:])
cph_auc, cph_mean_auc = cumulative_dynamic_auc(
    data_y, data_y_test, y_riskpred,time_points
)
#cph_aucs.loc[cv,time_points] = cph_auc
#cph_aucs.loc[cv,'mean'] = cph_mean_auc

In [30]:
for i in range(len(time_points)):
    if np.abs(cph_auc[i] - auc.iloc[0,i+2]) > 1e-6:
        print(cph_auc[i],auc.iloc[0,i+2],time_points[i])

In [18]:
true['y_risk'] = y_riskpred

In [19]:
true[['eid','Status','Survival_in_years','y_risk']].to_csv(f'{data_path}/{feature}/{diag}rsf_testrisk_CV{cv}.csv')

,eid,Status,Survival_in_years,Average0200___0259,mean_N_sedentary_intervals_07-23,mean_N_sedentary_intervals_per24h,Average0100___0159,Tuesday_average_acceleration,mean_imputed_hours_per24h,std_movement_during_sedentary,...,Average2100___2159,Average1800___1859,Average2200___2259,mean_N_MVPA_intervals_per24h,max_sedentary_hours_consecutive,Average0000___0059,mean_N_imputed_intervals_07-23,Average0300___0359,Intercept,y_risk
0,1085206,True,4.727225,-0.030563,0.347139,0.699216,-0.065808,0.594086,-0.520936,-0.176475,...,-0.613675,-0.209372,-0.289810,0.498167,-0.565700,-0.081009,-0.444342,-0.007875,1,2.501160
1,1090225,True,6.432941,-0.053512,-1.128361,-0.962776,-0.141940,2.184220,-0.344543,0.042039,...,-0.128409,0.253599,-0.302048,2.290737,-0.357729,-0.241184,-0.444342,-0.029705,1,0.114527
2,1142040,True,3.872998,-0.044852,-0.994224,-1.213213,-0.001887,0.333104,-0.450606,0.023129,...,-0.030638,0.503486,-0.569172,-0.859841,3.170076,-0.271292,-0.444342,0.000857,1,0.012924
3,1177770,True,2.263108,-0.048749,-0.357077,-0.689572,-0.178569,-0.249461,0.121112,-0.177278,...,0.361791,-0.384550,0.194221,-0.533919,0.266184,-0.346563,-0.444342,-0.077108,1,0.343585
4,1254917,True,2.761407,-0.017574,-0.424145,-0.666805,-0.123984,0.717709,-0.520936,-0.195622,...,-0.454013,0.102343,-0.591960,0.443847,-0.249892,-0.208667,-0.444342,-0.045298,1,0.615051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5015,6016647,False,5.814174,-0.040089,-0.323543,-0.279765,-0.138349,1.103931,-0.213524,0.176225,...,-0.128409,0.556113,0.458391,1.367292,0.119834,-0.268282,-0.444342,-0.050912,1,0.000000
5016,6017283,False,6.139985,-0.057408,-0.558281,-0.735106,-0.154150,0.295936,-0.307109,-0.139583,...,-0.203306,0.228942,0.373570,-0.642559,-0.373135,-0.280325,-0.444342,-0.061515,1,0.091180
5017,6019449,False,6.898385,0.007106,-0.088804,-0.097629,-0.087355,1.900614,-0.253227,0.148568,...,0.061751,0.026898,-0.155193,-0.153677,0.027402,-0.141225,-0.444342,-0.009746,1,0.000000
5018,6023481,False,7.358353,-0.020172,-0.189406,-0.370833,-0.131885,-0.319756,-0.520936,-0.143631,...,-0.353551,-0.061795,0.083658,-0.588239,0.027402,-0.309229,-0.444342,-0.073365,1,0.034851


# get new auprc metric from recomputed risk

In [8]:
data_path = '/scratch/c.c21013066/data/ukbiobank/analyses/survival/prodromal/noOsteo'
feature = 'intercept'
name = '_matched'
diag = 'diag_ProdHC'
cv = 0
time_points = np.arange(2.5, 7,0.1)
dt=dtype=[('Status', '?'), ('Survival_in_years', '<f8')]

In [9]:
# load preds
#preds = np.load(f'{data_path}/{feature}/rsf_{testpred_CV{cv}.csv.npy')
# load true
true = pd.read_csv(f'{data_path}/{feature}/{name}rsf_testrisk_CV{cv}.csv')
# auc
auc = pd.read_csv(f'{data_path}/{feature}/{name}rsf_aucs_5cv.csv')

merged = pd.read_csv(f'/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoMatchedHC.csv').set_index('eid')
prodage = pd.read_csv(f'/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoMatchedHC.csv').set_index('eid')
merged.loc[prodage[prodage['diag_ProdHC']==1].index,'acc_time_to_diagnosis'] = prodage.loc[prodage['diag_ProdHC']==1,'acc_time_to_diagnosis'].values

merged.loc[merged[diag]==0,'acc_time_to_diagnosis'] = (pd.Timestamp(datetime.datetime(2021,3,1)) - pd.to_datetime(merged.loc[merged[diag]==0,'date_accelerometry']) )/ np.timedelta64(1,'Y')
merged['Intercept'] = 1
merged = merged.dropna(subset=[diag])

train = merged[ ~merged.index.isin(true.eid) ]
train[diag] = train[diag].astype('?')
train['acc_time_to_diagnosis'] = train['acc_time_to_diagnosis'].astype('<f8')
train = train.drop(columns=['Status'])
train = train.rename(columns={diag:'Status','acc_time_to_diagnosis':'Survival_in_years'})

data_y = np.array([tuple(row) for row in train[['Status','Survival_in_years']].values], dtype=dt)

data_y_test = np.array([tuple(row) for row in true[['Status','Survival_in_years']].values], dtype=dt)



/tmp/ipykernel_111336/985211745.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[diag] = train[diag].astype('?')
/tmp/ipykernel_111336/985211745.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['acc_time_to_diagnosis'] = train['acc_time_to_diagnosis'].astype('<f8')


In [10]:

cph_auc, cph_mean_auc = cumulative_dynamic_auc(
    data_y, data_y_test, true['y_risk'],time_points
)

cph_auprc, cph_mean_auprc = cumulative_dynamic_auprc(
    data_y, data_y_test, true['y_risk'].astype(float),time_points
)
#cph_aucs.loc[cv,time_points] = cph_auc
#cph_aucs.loc[cv,'mean'] = cph_mean_auc

[0.         0.02173913 1.        ]
[0. 0. 1.]
[0.         0.04347826 1.        ]
[0. 0. 1.]
[0.         0.04347826 1.        ]
[0. 0. 1.]
[0.         0.06521739 1.        ]
[0. 0. 1.]
[0.         0.06521739 1.        ]
[0. 0. 1.]
[0.         0.08695652 1.        ]
[0. 0. 1.]
[0.         0.08695652 1.        ]
[0. 0. 1.]
[0.         0.08695652 1.        ]
[0. 0. 1.]
[0.         0.08695652 1.        ]
[0. 0. 1.]
[0.         0.08695652 1.        ]
[0. 0. 1.]
[0.         0.10869565 1.        ]
[0. 0. 1.]
[0.         0.10869565 1.        ]
[0. 0. 1.]
[0.         0.10869565 1.        ]
[0. 0. 1.]
[0.         0.10869565 1.        ]
[0. 0. 1.]
[0.         0.13043478 1.        ]
[0. 0. 1.]
[0.         0.13043478 1.        ]
[0. 0. 1.]
[0.         0.17391304 1.        ]
[0. 0. 1.]
[0.         0.19565217 1.        ]
[0. 0. 1.]
[0.         0.19565217 1.        ]
[0. 0. 1.]
[0.        0.2173913 1.       ]
[0. 0. 1.]
[0.        0.2173913 1.       ]
[0. 0. 1.]
[0.         0.23913043 1.        ]
[0. 0

In [11]:
from sklearn import metrics

In [12]:
recall = np.array([0.,0.,1.])
precision = np.array([0.,0.0231178,1.])
metrics.auc(recall[::-1],precision),np.trapz(precision,recall[::-1])

(0.0115589, -0.0115589)

In [13]:
cph_auprc,cph_mean_auprc

(array([0.01086957, 0.02173913, 0.02173913, 0.0326087 , 0.0326087 ,
        0.04347826, 0.04347826, 0.04347826, 0.04347826, 0.04347826,
        0.05434783, 0.05434783, 0.05434783, 0.05434783, 0.06521739,
        0.06521739, 0.08695652, 0.09782609, 0.09782609, 0.10869565,
        0.10869565, 0.11956522, 0.11956522, 0.14130435, 0.14130435,
        0.15217391, 0.17391304, 0.17391304, 0.20022971, 0.2145201 ,
        0.23849796, 0.25823351, 0.272082  , 0.28749998, 0.32188374,
        0.33936405, 0.34957389, 0.36041712, 0.37195456, 0.38425509,
        0.4186494 , 0.4186494 , 0.4186494 , 0.43273056, 0.44779192]),
 0.1895102682980278)